In [1]:
import os, json, shutil

src = "/home/mango/ERNIE/PaddleOCR-VL-SFT-test02"             # ERNIE 출력
dst = "/home/mango/ERNIE/PaddleOCR-VL-SFT-test02-hf"          # 변환 출력

os.makedirs(dst, exist_ok=True)

# 1) peft_model-*.safetensors -> adapter_model-*.safetensors
for fn in os.listdir(src):
    if fn.startswith("peft_model") and fn.endswith(".safetensors"):
        shutil.copy2(os.path.join(src, fn),
                     os.path.join(dst, fn.replace("peft_model", "adapter_model")))

# 2) 인덱스 파일도 이름 치환
idx_in = os.path.join(src, "peft_model.safetensors.index.json")
if os.path.exists(idx_in):
    with open(idx_in, "r") as f: idx = json.load(f)
    if "weight_map" in idx:
        idx["weight_map"] = {k: v.replace("peft_model", "adapter_model")
                             for k, v in idx["weight_map"].items()}
    with open(os.path.join(dst, "adapter_model.safetensors.index.json"), "w") as f:
        json.dump(idx, f, ensure_ascii=False, indent=2)

# 3) lora_config.json -> adapter_config.json (필수 키만 매핑)
with open(os.path.join(src, "lora_config.json"), "r") as f:
    lc = json.load(f)

adapter_cfg = {
    "peft_type": "LORA",
    "task_type": "CAUSAL_LM",
    "r": lc.get("lora_rank", 16),
    "lora_alpha": lc.get("lora_alpha", 32),
    "lora_dropout": lc.get("lora_dropout", 0.05),
    "bias": "none",
    "target_modules": lc.get("target_modules", []),
    "inference_mode": True
}
with open(os.path.join(dst, "adapter_config.json"), "w") as f:
    json.dump(adapter_cfg, f, ensure_ascii=False, indent=2)

print("HF PEFT 변환 완료:", dst)

HF PEFT 변환 완료: /home/mango/ERNIE/PaddleOCR-VL-SFT-test02-hf


In [ ]:
# pip install paddlepaddle paddleformers
import os, json, numpy as np
import paddle
from paddle import nn, tensor as T
from ernie.modeling_paddleocr_vl import PaddleOCRVLForConditionalGeneration
from ernie.tokenizer import Ernie4_5_Tokenizer
from data_processor.image_preprocessor.image_preprocessor_siglip import SiglipImageProcessor
from paddleformers.peft import LoRAModel

BASE = "PaddlePaddle/PaddleOCR-VL"                  # 학습에 사용한 베이스와 동일
LORA = "/home/mango/ERNIE/PaddleOCR-VL-SFT-test01"  # lora_config.json + peft_model-*.safetensors
OUT  = "/home/mango/ERNIE/PaddleOCR-VL-SFT-paddle-merged"
os.makedirs(OUT, exist_ok=True)

# 1) 베이스 로드 (HF→Paddle 변환 필요 시 convert_from_hf=True)
model = PaddleOCRVLForConditionalGeneration.from_pretrained(
    BASE, convert_from_hf=True
)
tok   = Ernie_4_5_Tokenizer = Ernie4_5_Tokenizer.from_pretrained(
    BASE, padding_side="right", model_max_length=4096
)
proc  = SiglipImageProcessor.from_pretrained(BASE)

# 2) LoRA 어댑터 적용(ERNIE 포맷에서 직접 로드)
model = LoRAModel.from_pretrained(model=model, lora_path=LORA)
model.eval()

# 3) LoRA 수동 병합: ΔW = (B @ A) * (alpha / r); base.W += ΔW
with open(os.path.join(LORA, "lora_config.json"), "r") as f:
    lcfg = json.load(f)
alpha = float(lcfg.get("lora_alpha", 16))
r     = float(lcfg.get("lora_rank", 16))
scale = alpha / max(r, 1.0)

state  = model.state_dict()
params = dict(model.named_parameters())

def _to_np(x: T.Variable): return np.array(x)

def add_inplace(param: nn.Parameter, delta_np: np.ndarray):
    base_np = _to_np(param)
    param.set_value(T.to_tensor(base_np + delta_np, dtype=param.dtype))

merged_cnt, skipped = 0, 0

def try_merge_pair(a_key, b_key):
    global merged_cnt, skipped
    if a_key not in state or b_key not in state:
        skipped += 1; return
    if a_key.endswith(".lora_A.weight"):
        base_key = a_key[:-len(".lora_A.weight")] + ".weight"
    elif a_key.endswith(".lora_down.weight"):
        base_key = a_key[:-len(".lora_down.weight")] + ".weight"
    else:
        skipped += 1; return
    if base_key not in params:
        skipped += 1; return
    A = state[a_key]   # [r, in]
    B = state[b_key]   # [out, r]
    try:
        delta = T.matmul(B, A) * scale
    except Exception:
        delta = T.to_tensor(_to_np(B) @ _to_np(A), dtype=B.dtype) * scale
    add_inplace(params[base_key], _to_np(delta))
    merged_cnt += 1

# 패턴1: lora_A/lora_B
for k in list(state.keys()):
    if k.endswith(".lora_A.weight"):
        try_merge_pair(k, k.replace(".lora_A.weight", ".lora_B.weight"))

# 패턴2: lora_down/lora_up
for k in list(state.keys()):
    if k.endswith(".lora_down.weight"):
        try_merge_pair(k, k.replace(".lora_down.weight", ".lora_up.weight"))

print(f"[LoRA merge] merged={merged_cnt}, skipped={skipped}, scale={scale}")

# (선택) 어댑터 파라미터 0으로 막기(중복 적용 방지)
for name, p in params.items():
    if name.endswith(("lora_A.weight","lora_B.weight","lora_down.weight","lora_up.weight")):
        p.set_value(T.zeros_like(p))

# 4) 병합된 베이스 가중치 저장 (용량이 커져야 정상)
model.save_pretrained(OUT)
tok.save_pretrained(OUT)
proc.save_pretrained(OUT)
print("병합 완료:", OUT)

/root/anaconda3/envs/deepseek-ocr/lib/python3.12/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
W1110 22:36:10.879169 22843 gpu_resources.cc:114] Please NOTE: device: 0, GPU Compute Capability: 8.6, Driver API Version: 12.6, Runtime API Version: 11.8
[2025-11-10 22:36:12,904] [    INFO] - Using download source: huggingface
[2025-11-10 22:36:12,904] [    INFO] - Loading configuration file PaddlePaddle/PaddleOCR-VL/config.json
[2025-11-10 22:36:12,906] [    INFO] - Loading weights file PaddlePaddle/PaddleOCR-VL/model.safetensors
[2025-11-10 22:36:17,476] [    INFO] - Loaded weights file from disk, setting weights to model.
[2025-11-10 22:36:17,636] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-10 22:36:17,640] [    INF

[LoRA merge] merged=0, skipped=0, scale=1.0
No pairs merged. Check LoRA keys sample: ['model.model.layers.0.self_attn.o_proj.lora_A', 'model.model.layers.0.self_attn.o_proj.lora_B', 'model.model.layers.0.mlp.down_proj.lora_A', 'model.model.layers.0.mlp.down_proj.lora_B', 'model.model.layers.1.self_attn.o_proj.lora_A', 'model.model.layers.1.self_attn.o_proj.lora_B', 'model.model.layers.1.mlp.down_proj.lora_A', 'model.model.layers.1.mlp.down_proj.lora_B', 'model.model.layers.2.self_attn.o_proj.lora_A', 'model.model.layers.2.self_attn.o_proj.lora_B']


[2025-11-10 22:36:21,876] [    INFO] - Model weights saved in /home/mango/ERNIE/PaddleOCR-VL-SFT-paddle-merged/model_state.pdparams
[2025-11-10 22:36:21,884] [    INFO] - tokenizer config file saved in /home/mango/ERNIE/PaddleOCR-VL-SFT-paddle-merged/tokenizer_config.json
[2025-11-10 22:36:21,884] [    INFO] - Special tokens file saved in /home/mango/ERNIE/PaddleOCR-VL-SFT-paddle-merged/special_tokens_map.json
[2025-11-10 22:36:21,885] [    INFO] - added tokens file saved in /home/mango/ERNIE/PaddleOCR-VL-SFT-paddle-merged/added_tokens.json
[2025-11-10 22:36:21,887] [    INFO] - Image processor saved in /home/mango/ERNIE/PaddleOCR-VL-SFT-paddle-merged/preprocessor_config.json


Saved to: /home/mango/ERNIE/PaddleOCR-VL-SFT-paddle-merged  total size (MB): 1832.11


In [4]:
# ERNIE(Paddle) 방식: 베이스 로드 → LoRA 어댑터(체크포인트) 적용 → 저장
# pip install paddlepaddle paddlefsl paddleformers Pillow

import os
from ernie.modeling_paddleocr_vl import PaddleOCRVLForConditionalGeneration
from ernie.tokenizer import Ernie4_5_Tokenizer
from data_processor.image_preprocessor.image_preprocessor_siglip import SiglipImageProcessor
from paddleformers.peft import LoRAModel

# 1) 경로 설정
BASE_MODEL_ID = "PaddlePaddle/PaddleOCR-VL"  # 학습 시 사용한 베이스와 동일해야 함
LORA_ROOT_DIR = "/home/mango/ERNIE/PaddleOCR-VL-SFT-test04"   # peft_model-*.safetensors, lora_config.json 이 있는 루트
SAVE_DIR      = "/home/mango/ERNIE/PaddleOCR-VL-SFT-paddle-merged"
os.makedirs(SAVE_DIR, exist_ok=True)

# 2) 베이스 모델/토크나이저/이미지프로세서 로드 (HF→Paddle 변환 필요시 convert_from_hf=True)
model = PaddleOCRVLForConditionalGeneration.from_pretrained(
    BASE_MODEL_ID,
    convert_from_hf=True,  # HF 허브 베이스를 Paddle로 변환 로드
)
tokenizer = Ernie4_5_Tokenizer.from_pretrained(
    BASE_MODEL_ID, padding_side="right", model_max_length=4096
)
image_proc = SiglipImageProcessor.from_pretrained(BASE_MODEL_ID)

# 3) LoRA 어댑터 적용(체크포인트 루트에서 직접 로드)
#   ERNIE 포맷: LORA_ROOT_DIR 안에 lora_config.json + peft_model-*.safetensors 가 있어야 합니다.
model = LoRAModel.from_pretrained(model=model, lora_path=LORA_ROOT_DIR)
model.eval()

# (선택) 병합 API가 있을 경우 병합 저장
if hasattr(model, "merge_and_unload"):
    try:
        model = model.merge_and_unload()
        print("LoRA merged into base weights.")
    except Exception as e:
        print(f"merge_and_unload 미지원/실패: {e}; 어댑터 래핑 상태로 저장합니다.")

# 4) 저장(ERNIE/Paddle 포맷)
model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)
image_proc.save_pretrained(SAVE_DIR)
print("Paddle(ERNIE) 저장 완료:", SAVE_DIR)

[2025-11-11 09:48:31,797] [    INFO] - Using download source: huggingface
[2025-11-11 09:48:31,799] [    INFO] - Loading configuration file PaddlePaddle/PaddleOCR-VL/config.json
[2025-11-11 09:48:31,801] [    INFO] - Loading weights file PaddlePaddle/PaddleOCR-VL/model.safetensors
[2025-11-11 09:48:35,200] [    INFO] - Loaded weights file from disk, setting weights to model.
[2025-11-11 09:48:35,370] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-11 09:48:35,373] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-11 09:48:35,381] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-11 09:48:35,387] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-11 09:48:35,392] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-11 09:48:35,399] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-11 09:48:35,405] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-11 0

Paddle(ERNIE) 저장 완료: /home/mango/ERNIE/PaddleOCR-VL-SFT-paddle-merged


In [1]:
import paddle
from ernie.modeling_paddleocr_vl import PaddleOCRVLForConditionalGeneration
from ernie.tokenizer import Ernie4_5_Tokenizer
from data_processor.image_preprocessor.image_preprocessor_siglip import SiglipImageProcessor
from paddleformers.peft import LoRAModel

# 1) 경로 설정
base_model = "PaddlePaddle/PaddleOCR-VL"  # 베이스 모델 허브 경로(또는 로컬 경로)
lora_dir   = "/home/mango/ERNIE/PaddleOCR-VL-SFT-test04/checkpoint-20/"  # peft_model*.safetensors 있는 디렉터리

# 2) 베이스 모델/토크나이저/이미지 프로세서 로드
model = PaddleOCRVLForConditionalGeneration.from_pretrained(base_model, convert_from_hf=True)
tokenizer = Ernie4_5_Tokenizer.from_pretrained(base_model, padding_side="right", model_max_length=8192)
image_preprocessor = SiglipImageProcessor.from_pretrained(base_model)

# 3) LoRA 어댑터 적용 (출력 디렉터리에서)
model = LoRAModel.from_pretrained(model=model, lora_path=lora_dir)
model.eval()    
model.mark_only_lora_as_trainable()
model.print_trainable_parameters()
# (선택) LoRA 병합 후 고정이 필요하면, 다음 API가 지원되면 사용:
# model.merge_and_unload()  # 지원되는 경우에만

/root/anaconda3/envs/deepseek-ocr/lib/python3.12/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
W1111 10:18:47.659309 17906 gpu_resources.cc:114] Please NOTE: device: 0, GPU Compute Capability: 8.6, Driver API Version: 12.6, Runtime API Version: 11.8
[2025-11-11 10:18:49,703] [    INFO] - Using download source: huggingface
[2025-11-11 10:18:49,704] [    INFO] - Loading configuration file PaddlePaddle/PaddleOCR-VL/config.json
[2025-11-11 10:18:49,705] [    INFO] - Loading weights file PaddlePaddle/PaddleOCR-VL/model.safetensors
[2025-11-11 10:18:53,727] [    INFO] - Loaded weights file from disk, setting weights to model.
[2025-11-11 10:18:53,866] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-11 10:18:53,869] [    INF

In [5]:
# from PIL import Image
# import torch
# from transformers import AutoModelForCausalLM, AutoProcessor

# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# CHOSEN_TASK = "ocr"  # Options: 'ocr' | 'table' | 'chart' | 'formula'
# PROMPTS = {
#     "ocr": "OCR:",
#     "table": "Table Recognition:",
#     "formula": "Formula Recognition:",
#     "chart": "Chart Recognition:",
# }

# image_path = "000000014_경남.jpg"
# image = Image.open(image_path).convert("RGB")

# processor = AutoProcessor.from_pretrained(base_model)

# messages = [
#     {"role": "user",         
#      "content": [
#             {"type": "image", "image": image},
#             {"type": "text", "text": PROMPTS[CHOSEN_TASK]},
#         ]
#     }
# ]
# inputs = processor.apply_chat_template(
#     messages, 
#     tokenize=True, 
#     add_generation_prompt=True, 	
#     return_dict=True,
#     return_tensors="pt"
# ).to(DEVICE)

# outputs = model.infer(**inputs, max_new_tokens=1024)
# outputs = processor.batch_decode(outputs, skip_special_tokens=True)[0]
# print(outputs)


In [1]:
import paddle
from ernie.modeling_paddleocr_vl import PaddleOCRVLForConditionalGeneration
from ernie.tokenizer import Ernie4_5_Tokenizer
from data_processor.image_preprocessor.image_preprocessor_siglip import SiglipImageProcessor
from paddleformers.peft import LoRAModel

BASE = "PaddlePaddle/PaddleOCR-VL"                     # 학습 베이스와 동일
LORA = "/home/mango/ERNIE/PaddleOCR-VL-SFT-test02"     # LoRA 출력 디렉터리(ERNIE 포맷)

# 1) 베이스 로드(HF->Paddle 변환 필요 시)
model = PaddleOCRVLForConditionalGeneration.from_pretrained(BASE, convert_from_hf=True)
tokenizer = Ernie4_5_Tokenizer.from_pretrained(LORA, padding_side="right", model_max_length=4096)
image_preprocess = SiglipImageProcessor.from_pretrained(LORA)

# 2) LoRA 어댑터 적용
model = LoRAModel.from_pretrained(model=model, lora_path=LORA)
model.eval()

/root/anaconda3/envs/deepseek-ocr/lib/python3.12/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
W1111 13:29:50.981730 23682 gpu_resources.cc:114] Please NOTE: device: 0, GPU Compute Capability: 8.6, Driver API Version: 12.6, Runtime API Version: 11.8
[2025-11-11 13:29:53,141] [    INFO] - Using download source: huggingface
[2025-11-11 13:29:53,143] [    INFO] - Loading configuration file PaddlePaddle/PaddleOCR-VL/config.json
[2025-11-11 13:29:53,145] [    INFO] - Loading weights file PaddlePaddle/PaddleOCR-VL/model.safetensors
[2025-11-11 13:29:57,286] [    INFO] - Loaded weights file from disk, setting weights to model.
[2025-11-11 13:29:57,449] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-11 13:29:57,453] [    INF

In [1]:
from paddlex.inference.models.doc_vlm.processors.paddleocr_vl._paddleocr_vl import PaddleOCRVLProcessor
from data_processor.image_preprocessor.image_preprocessor_siglip import SiglipImageProcessor
from ernie.tokenizer import Ernie4_5_Tokenizer  # paddlex 배포 토크나이저가 HF 포맷

model_dir = "./PaddleOCR-VL-SFT-test03"  # --vl_rec_model_dir와 동일한 구조
image_processor = SiglipImageProcessor.from_pretrained(model_dir)
tokenizer = Ernie4_5_Tokenizer.from_pretrained(model_dir, trust_remote_code=True)

processor = PaddleOCRVLProcessor(image_processor=image_processor, tokenizer=tokenizer)
# 이후 processor.preprocess(...) / processor.postprocess(...) 사용

/root/anaconda3/envs/deepseek-ocr/lib/python3.12/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
[2025-11-11 14:23:12,202] [ WARNING] - PretrainedTokenizer will be deprecated and removed in the next major release. Please migrate to Hugging Face's transformers.PreTrainedTokenizer. Checkout paddleformers/transformers/qwen/tokenizer.py for an example: use class QWenTokenizer(PaddleTokenizerMixin, hf.PreTrainedTokenizer) to support multisource download and Paddle tokenizer operations.


In [2]:
processor

In [1]:
from PIL import Image
import torch
from transformers import AutoModelForCausalLM, AutoProcessor

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

CHOSEN_TASK = "ocr"  # Options: 'ocr' | 'table' | 'chart' | 'formula'
PROMPTS = {
    "ocr": "OCR:",
    "table": "Table Recognition:",
    "formula": "Formula Recognition:",
    "chart": "Chart Recognition:",
}

model_path = "PaddlePaddle/PaddleOCR-VL"
image_path = "000000014_경남.jpg"
image = Image.open(image_path).convert("RGB")

model = AutoModelForCausalLM.from_pretrained(
    model_path, trust_remote_code=True, torch_dtype=torch.bfloat16
).to(DEVICE).eval()
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)

messages = [
    {"role": "user",         
     "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": PROMPTS[CHOSEN_TASK]},
        ]
    }
]
inputs = processor.apply_chat_template(
    messages, 
    tokenize=True, 
    add_generation_prompt=True, 	
    return_dict=True,
    return_tensors="pt"
).to(DEVICE)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:
inputs

{'input_ids': tensor([[100273,   2969,  93963,  93919, 101305, 100295, 100295, 100295, 100295,
         100295, 100295, 100295, 100295, 100295, 100295, 100295, 100295, 100295,
         100295, 100295, 100295, 100295, 100295, 100295, 100295, 100295, 100295,
         100295, 100295, 100295, 100295, 100295, 100295, 100295, 100295, 100295,
         100295, 100295, 100295, 100295, 100295, 100295, 100295, 100295, 100295,
         100295, 100295, 100295, 100295, 100295, 100295, 100295, 100295, 100295,
         100295, 100295, 100295, 100295, 100295, 100295, 100295, 100295, 100295,
         100295, 100295, 100295, 100295, 100295, 100295, 100295, 100295, 100295,
         100295, 100295, 100295, 100295, 100295, 100295, 100295, 100295, 100295,
         100295, 100295, 100295, 100295, 100295, 100295, 100295, 100295, 100295,
         100295, 100295, 100295, 100295, 100295, 100295, 100295, 100295, 100295,
         100295, 100295, 100295, 100295, 100295, 100295, 100295, 100295, 100295,
         10029

In [7]:
import paddle
from ernie.modeling_paddleocr_vl import PaddleOCRVLForConditionalGeneration
from ernie.tokenizer import Ernie4_5_Tokenizer
from data_processor.image_preprocessor.image_preprocessor_siglip import SiglipImageProcessor
from paddleformers.peft import LoRAModel
from paddleformers.peft import LoRAConfig, LoRAModel, PrefixConfig, PrefixModelForCausalLM
from paddleformers.transformers import AutoModelForCausalLM, AutoTokenizer
from paddleformers.peft import LoRAAutoModel
BASE = "PaddlePaddle/PaddleOCR-VL"                     # 학습 베이스와 동일
LORA = "/home/mango/ERNIE/PaddleOCR-VL-SFT-test06"     # LoRA 출력 디렉터리(ERNIE 포맷)

# 1) 베이스 로드(HF->Paddle 변환 필요 시)
base_model = PaddleOCRVLForConditionalGeneration.from_pretrained(BASE, convert_from_hf=True)
tokenizer = Ernie4_5_Tokenizer.from_pretrained(LORA, padding_side="right", model_max_length=4096)
image_preprocess = SiglipImageProcessor.from_pretrained(LORA)

#lora_config = LoRAConfig.from_pretrained(LORA)
#dtype = lora_config.dtype
#lora_config.merge_weights = True
# 2) LoRA 어댑터 적용

lora_model = LoRAModel.from_pretrained(base_model, lora_path=LORA)
lora_model.merge()
lora_model.restore_original_model()
if hasattr(lora_model, "model"):
    # LoRAModel이 감싸고 있는 실제 모델 객체 접근
    merged_base_model = lora_model.model
    #print(merged_base_model)
    merged_base_model.save_pretrained(
    LORA,
    safe_serialization=True,   # HF safetensors로 저장
)
#lora_model.model.save_pretrained(LORA)

[2025-11-25 16:08:25,515] [    INFO] - Using download source: huggingface
[2025-11-25 16:08:25,518] [    INFO] - Loading configuration file PaddlePaddle/PaddleOCR-VL/config.json
[2025-11-25 16:08:25,522] [    INFO] - Loading weights file PaddlePaddle/PaddleOCR-VL/model.safetensors
[2025-11-25 16:08:33,296] [    INFO] - Loaded weights file from disk, setting weights to model.
[2025-11-25 16:08:33,409] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 16:08:33,413] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 16:08:33,417] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 16:08:33,422] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 16:08:33,425] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 16:08:33,428] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 16:08:33,431] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 1

In [10]:
import os
try:
    from safetensors import safe_open
    from safetensors.torch import save_file
    import torch 
except ImportError:
    print("❌ 'pip install safetensors torch' 필요")
    exit()

# =========================================================
# [설정] 문제가 있는 LoRA 모델 경로
# =========================================================
TARGET_FILE = "/home/mango/ERNIE/PaddleOCR-VL-SFT-test06/model.safetensors"
# =========================================================

def fix_transposed_weights():
    print(f"📂 파일 로드 중: {TARGET_FILE}")
    
    if not os.path.exists(TARGET_FILE):
        print("❌ 파일 없음")
        return

    tensors_to_save = {}
    fixed_count = 0
    
    with safe_open(TARGET_FILE, framework="pt", device="cpu") as f:
        for key in f.keys():
            tensor = f.get_tensor(key)
            
            # 2D 텐서(Linear Weight)인 경우 전치 대상인지 확인
            if len(tensor.shape) == 2:
                # Transpose 대상 키워드 필터링
                if "proj.weight" in key or "fc1.weight" in key or "fc2.weight" in key or "lm_head.weight" in key or "linear" in key:
                    print(f"🔧 Transposing: {key} {tensor.shape} -> {tensor.t().shape}")
                    
                    # [핵심 수정] .t() 뒤에 .contiguous() 추가
                    tensor = tensor.t().contiguous() 
                    
                    fixed_count += 1
            
            tensors_to_save[key] = tensor

    # 3. 저장
    if fixed_count > 0:
        print(f"\n💾 총 {fixed_count}개의 텐서를 뒤집어 저장합니다...")
        output_path = TARGET_FILE.replace(".safetensors", "_fixed.safetensors")
        
        # 메타데이터 없이 저장 (순수 텐서 데이터)
        save_file(tensors_to_save, output_path)
        
        print(f"✅ 저장 완료: {output_path}")
        print("👉 이 파일의 이름을 'model.safetensors'로 변경해서 사용하세요!")
    else:
        print("⚠️ 수정할 텐서를 찾지 못했습니다.")

if __name__ == "__main__":
    fix_transposed_weights()

📂 파일 로드 중: /home/mango/ERNIE/PaddleOCR-VL-SFT-test06/model.safetensors
🔧 Transposing: lm_head.weight torch.Size([1024, 103424]) -> torch.Size([103424, 1024])
🔧 Transposing: mlp_AR.linear_1.weight torch.Size([4608, 4608]) -> torch.Size([4608, 4608])
🔧 Transposing: mlp_AR.linear_2.weight torch.Size([4608, 1024]) -> torch.Size([1024, 4608])
🔧 Transposing: model.layers.0.mlp.down_proj.weight torch.Size([3072, 1024]) -> torch.Size([1024, 3072])
🔧 Transposing: model.layers.0.mlp.gate_proj.weight torch.Size([1024, 3072]) -> torch.Size([3072, 1024])
🔧 Transposing: model.layers.0.mlp.up_proj.weight torch.Size([1024, 3072]) -> torch.Size([3072, 1024])
🔧 Transposing: model.layers.0.self_attn.k_proj.weight torch.Size([1024, 256]) -> torch.Size([256, 1024])
🔧 Transposing: model.layers.0.self_attn.o_proj.weight torch.Size([2048, 1024]) -> torch.Size([1024, 2048])
🔧 Transposing: model.layers.0.self_attn.q_proj.weight torch.Size([1024, 2048]) -> torch.Size([2048, 1024])
🔧 Transposing: model.layers.0.s

In [ ]:
import os
try:
    from safetensors import safe_open
    from safetensors.torch import save_file
    import torch 
except ImportError:
    print("❌ 'pip install safetensors torch' 필요")
    exit()

# =========================================================
# [설정] 문제가 있는 LoRA 모델 경로
# =========================================================
TARGET_FILE = "/home/mango/ERNIE/PaddleOCR-VL-SFT-test06/model.safetensors"
# =========================================================

def fix_transposed_weights():
    print(f"📂 파일 로드 중: {TARGET_FILE}")
    
    if not os.path.exists(TARGET_FILE):
        print("❌ 파일 없음")
        return

    tensors_to_save = {}
    fixed_count = 0
    
    with safe_open(TARGET_FILE, framework="pt", device="cpu") as f:
        for key in f.keys():
            tensor = f.get_tensor(key)
            
            # 2D 텐서(Linear Weight)인 경우 전치 대상인지 확인
            if len(tensor.shape) == 2:
                # Transpose 대상 키워드 필터링
                if "proj.weight" in key or "fc1.weight" in key or "fc2.weight" in key or "lm_head.weight" in key or "linear" in key:
                    print(f"🔧 Transposing: {key} {tensor.shape} -> {tensor.t().shape}")
                    
                    # [핵심 수정] .t() 뒤에 .contiguous() 추가
                    tensor = tensor.t().contiguous() 
                    
                    fixed_count += 1
            
            tensors_to_save[key] = tensor

    # 3. 저장
    if fixed_count > 0:
        print(f"\n💾 총 {fixed_count}개의 텐서를 뒤집어 저장합니다...")
        output_path = TARGET_FILE.replace(".safetensors", "_fixed.safetensors")
        
        # 메타데이터 없이 저장 (순수 텐서 데이터)
        save_file(tensors_to_save, output_path)
        
        print(f"✅ 저장 완료: {output_path}")
        print("👉 이 파일의 이름을 'model.safetensors'로 변경해서 사용하세요!")
    else:
        print("⚠️ 수정할 텐서를 찾지 못했습니다.")

if __name__ == "__main__":
    fix_transposed_weights()

📂 파일 로드 중: /home/mango/ERNIE/PaddleOCR-VL-SFT-test06/model.safetensors
🔧 Transposing: lm_head.weight torch.Size([1024, 103424]) -> torch.Size([103424, 1024])
🔧 Transposing: mlp_AR.linear_1.weight torch.Size([4608, 4608]) -> torch.Size([4608, 4608])
🔧 Transposing: mlp_AR.linear_2.weight torch.Size([4608, 1024]) -> torch.Size([1024, 4608])
🔧 Transposing: model.layers.0.mlp.down_proj.weight torch.Size([3072, 1024]) -> torch.Size([1024, 3072])
🔧 Transposing: model.layers.0.mlp.gate_proj.weight torch.Size([1024, 3072]) -> torch.Size([3072, 1024])
🔧 Transposing: model.layers.0.mlp.up_proj.weight torch.Size([1024, 3072]) -> torch.Size([3072, 1024])
🔧 Transposing: model.layers.0.self_attn.k_proj.weight torch.Size([1024, 256]) -> torch.Size([256, 1024])
🔧 Transposing: model.layers.0.self_attn.o_proj.weight torch.Size([2048, 1024]) -> torch.Size([1024, 2048])
🔧 Transposing: model.layers.0.self_attn.q_proj.weight torch.Size([1024, 2048]) -> torch.Size([2048, 1024])
🔧 Transposing: model.layers.0.s

In [11]:
import os
try:
    from safetensors import safe_open
except ImportError:
    print("❌ 'safetensors' 라이브러리가 필요합니다. (pip install safetensors)")
    exit()

# =========================================================
# [설정] 정확한 파일 경로 (safetensors 파일까지 포함)
# =========================================================
# 예: /home/user/.cache/huggingface/.../model.safetensors
BASE_FILE = "/home/mango/ERNIE/PaddlePaddle/PaddleOCR-VL/model.safetensors" 

# 예: /home/mango/ERNIE/PaddleOCR-VL-SFT-test06/model.safetensors
LORA_FILE = "/home/mango/ERNIE/PaddleOCR-VL-SFT-test06/model_fixed.safetensors"
# =========================================================

def load_safetensors_meta(path):
    if not os.path.exists(path):
        print(f"❌ 파일 없음: {path}")
        return None
    
    meta = {}
    try:
        with safe_open(path, framework="pt", device="cpu") as f:
            for key in f.keys():
                # 텐서 자체를 로드하지 않고 shape만 가져오고 싶지만,
                # safe_open은 get_tensor() 해야 shape을 알 수 있는 경우가 많음.
                # (메타데이터만 빠르게 보려면 f.get_slice(key).get_shape() 사용)
                tensor_slice = f.get_slice(key)
                meta[key] = tensor_slice.get_shape()
    except Exception as e:
        print(f"❌ 읽기 오류 ({path}): {e}")
        return None
        
    return meta

def compare_safetensors():
    print("🔍 Safetensors 메타데이터 추출 중...")
    
    base_meta = load_safetensors_meta(BASE_FILE)
    lora_meta = load_safetensors_meta(LORA_FILE)
    
    if not base_meta or not lora_meta:
        return

    base_keys = set(base_meta.keys())
    lora_keys = set(lora_meta.keys())

    print(f"\n📊 통계:")
    print(f"   - Base Keys: {len(base_keys)}")
    print(f"   - LoRA Keys: {len(lora_keys)}")

    # 1. 누락 (Missing)
    missing = base_keys - lora_keys
    print("\n" + "="*60)
    print(f"🚨 [1] 누락된 키 (Base O, LoRA X) - 총 {len(missing)}개")
    print("="*60)
    if missing:
        for k in sorted(list(missing)):
            print(f"   ➖ {k}")
    else:
        print("   ✅ 없음 (Good)")

    # 2. 불필요 (Extra)
    extra = lora_keys - base_keys
    print("\n" + "="*60)
    print(f"🚨 [2] 불필요한 키 (Base X, LoRA O) - 총 {len(extra)}개")
    print("="*60)
    if extra:
        for k in sorted(list(extra)):
            print(f"   ➕ {k}")
    else:
        print("   ✅ 없음 (Good)")

    # 3. Shape 불일치
    print("\n" + "="*60)
    print("📏 [3] Shape 불일치")
    print("="*60)
    common = base_keys & lora_keys
    mismatch_cnt = 0
    for k in sorted(list(common)):
        if base_meta[k] != lora_meta[k]:
            print(f"   ⚠️ {k}")
            print(f"       Base: {base_meta[k]}  vs  LoRA: {lora_meta[k]}")
            mismatch_cnt += 1
            
    if mismatch_cnt == 0:
        print("   ✅ Shape 모두 일치 (Good)")
    else:
        print(f"   ❌ 총 {mismatch_cnt}개 Shape 불일치 발견")

if __name__ == "__main__":
    compare_safetensors()

🔍 Safetensors 메타데이터 추출 중...

📊 통계:
   - Base Keys: 620
   - LoRA Keys: 620

🚨 [1] 누락된 키 (Base O, LoRA X) - 총 0개
   ✅ 없음 (Good)

🚨 [2] 불필요한 키 (Base X, LoRA O) - 총 0개
   ✅ 없음 (Good)

📏 [3] Shape 불일치
   ✅ Shape 모두 일치 (Good)


In [4]:
# 설정
BASE_MODEL_PATH = "PaddlePaddle/PaddleOCR-VL"   # Base 모델 폴더명 (또는 HuggingFace Hub ID)
OUTPUT_TXT = "base_model_dump.txt"

def dump_base_structure():
    print(f"🚀 Base 모델 로드 중: {BASE_MODEL_PATH}")
    try:
        model = PaddleOCRVLForConditionalGeneration.from_pretrained(BASE_MODEL_PATH, convert_from_hf=True)
    except Exception as e:
        print(f"❌ 모델 로드 실패: {e}")
        return

    state_dict = model.state_dict()
    print(f"✅ 로드 완료. ({len(state_dict)} keys)")
    
    with open(OUTPUT_TXT, "w", encoding="utf-8") as f:
        f.write(f"=== Base Model Dump ({len(state_dict)} keys) ===\n")
        f.write(f"Source: {BASE_MODEL_PATH}\n\n")
        
        sorted_keys = sorted(state_dict.keys())
        
        for key in sorted_keys:
            tensor = state_dict[key]
            # Shape과 Dtype 기록
            info = f"{key:<60} | Shape: {str(tensor.shape):<15} | Dtype: {tensor.dtype}\n"
            f.write(info)
            
            # 비교를 위해 head 부분은 콘솔에도 출력
            if "head" in key and "attention" in key:
                print(f"👀 {info.strip()}")
                
    print(f"\n✅ Base 모델 구조가 '{OUTPUT_TXT}' 파일에 저장되었습니다.")

if __name__ == "__main__":
    dump_base_structure()

[2025-11-25 14:52:52,329] [    INFO] - Using download source: huggingface
[2025-11-25 14:52:52,330] [    INFO] - Loading configuration file PaddlePaddle/PaddleOCR-VL/config.json
[2025-11-25 14:52:52,332] [    INFO] - Loading weights file PaddlePaddle/PaddleOCR-VL/model.safetensors


🚀 Base 모델 로드 중: PaddlePaddle/PaddleOCR-VL


[2025-11-25 14:52:55,754] [    INFO] - Loaded weights file from disk, setting weights to model.
[2025-11-25 14:52:55,804] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:52:55,806] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:52:55,808] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:52:55,813] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:52:55,815] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:52:55,817] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:52:55,820] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:52:55,822] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:52:55,824] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:52:55,827] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:52:55,831] [    INF

✅ 로드 완료. (620 keys)
👀 visual.vision_model.head.attention.in_proj_bias              | Shape: [3456]          | Dtype: paddle.bfloat16
👀 visual.vision_model.head.attention.in_proj_weight            | Shape: [3456, 1152]    | Dtype: paddle.bfloat16
👀 visual.vision_model.head.attention.out_proj.bias             | Shape: [1152]          | Dtype: paddle.bfloat16
👀 visual.vision_model.head.attention.out_proj.weight           | Shape: [1152, 1152]    | Dtype: paddle.bfloat16

✅ Base 모델 구조가 'base_model_dump.txt' 파일에 저장되었습니다.


In [2]:
# 경로 설정
BASE_MODEL_PATH = BASE
LORA_PATH = LORA  # 실제 LoRA 경로로 수정하세요

def print_layer_info(model, layer_name_keyword="q_proj"):
    """
    모델 내에서 특정 키워드가 포함된 첫 번째 레이어를 찾아 구조를 출력합니다.
    보통 LoRA는 Attention의 q_proj, v_proj 등에 적용되므로 이를 확인합니다.
    """
    found = False
    print(f"\n{'='*20} [{layer_name_keyword}] 레이어 구조 분석 {'='*20}")
    
    for name, sublayer in model.named_sublayers():
        if layer_name_keyword in name:
            print(f"📍 레이어 이름: {name}")
            print(f"🧩 레이어 타입: {type(sublayer)}")
            print(f"🔑 포함된 파라미터(Keys):")
            for param_name, param in sublayer.named_parameters():
                print(f"   - {param_name:<25} | Shape: {param.shape}")
            found = True
            break
            
    if not found:
        print(f"'{layer_name_keyword}'가 포함된 레이어를 찾지 못했습니다.")

def compare_structures():
    # 1. Base Model 로드 및 구조 확인
    print("\n🔵 [1] Base Model (원본) 로드 중...")
    base_model = PaddleOCRVLForConditionalGeneration.from_pretrained(BASE_MODEL_PATH, convert_from_hf=True)
    
    # 예시로 'q_proj' (Query Projection) 레이어를 뜯어봅니다.
    print_layer_info(base_model, "linear") # 혹은 'linear' 등
    
    # 2. LoRA Model 적용 및 구조 확인
    print("\n\n🟠 [2] LoRA Model (래핑 후) 로드 중...")
    # LoRAModel.from_pretrained는 내부적으로 base_model을 변형(In-place modification)합니다.
    lora_model = PaddleOCRVLForConditionalGeneration.from_pretrained(LORA_PATH)
    
    # 동일한 레이어가 어떻게 변했는지 확인
    print_layer_info(lora_model, "linear")


if __name__ == "__main__":
    compare_structures()

[2025-11-25 14:25:13,215] [    INFO] - Using download source: huggingface
[2025-11-25 14:25:13,216] [    INFO] - Loading configuration file PaddlePaddle/PaddleOCR-VL/config.json
[2025-11-25 14:25:13,217] [    INFO] - Loading weights file PaddlePaddle/PaddleOCR-VL/model.safetensors



🔵 [1] Base Model (원본) 로드 중...


[2025-11-25 14:25:16,562] [    INFO] - Loaded weights file from disk, setting weights to model.
[2025-11-25 14:25:16,685] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:25:16,688] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:25:16,692] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:25:16,698] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:25:16,702] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:25:16,707] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:25:16,712] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:25:16,719] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:25:16,723] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:25:16,726] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:25:16,734] [    INF


==================== [linear] 레이어 구조 분석 ====================
📍 레이어 이름: mlp_AR.linear_1
🧩 레이어 타입: <class 'paddle.nn.layer.common.Linear'>
🔑 포함된 파라미터(Keys):
   - weight                    | Shape: [4608, 4608]
   - bias                      | Shape: [4608]


🟠 [2] LoRA Model (래핑 후) 로드 중...


[2025-11-25 14:25:18,284] [    INFO] - Loaded weights file from disk, setting weights to model.
[2025-11-25 14:25:18,402] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:25:18,405] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:25:18,409] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:25:18,414] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:25:18,419] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:25:18,423] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:25:18,427] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:25:18,434] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:25:18,438] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:25:18,442] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-25 14:25:18,448] [    INF


==================== [linear] 레이어 구조 분석 ====================
📍 레이어 이름: mlp_AR.linear_1
🧩 레이어 타입: <class 'paddle.nn.layer.common.Linear'>
🔑 포함된 파라미터(Keys):
   - weight                    | Shape: [4608, 4608]
   - bias                      | Shape: [4608]


In [3]:
import paddle
import os
import json

# 경로 설정
BASE_MODEL_DIR = BASE
TARGET_MODEL_DIR = "./PaddleOCR-VL-SFT-test06"

def find_weight_file(directory):
    """폴더 내에서 가중치 파일을 찾습니다."""
    candidates = ["model_state.pdparams", "model.pdparams", "model.safetensors"]
    for fname in candidates:
        full_path = os.path.join(directory, fname)
        if os.path.exists(full_path):
            return full_path
    return None

def load_keys(path):
    if not path:
        return None, None
    
    print(f"📂 파일 로딩 시도: {path}")
    try:
        # 확장자에 따라 로드 방식 분기
        if path.endswith(".safetensors"):
            from safetensors.numpy import load_file
            # safetensors는 numpy dict로 반환됨
            state_dict = load_file(path)
            # 텐서로 변환이 필요할 수 있으나 키 비교 목적이면 그대로 사용 가능
        else:
            state_dict = paddle.load(path)
            
        return set(state_dict.keys()), state_dict
    except Exception as e:
        print(f"❌ 로드 에러: {e}")
        return None, None

def analyze_diff():
    print(f"🔍 분석 시작")
    
    # 파일 찾기
    base_file = find_weight_file(BASE_MODEL_DIR)
    target_file = find_weight_file(TARGET_MODEL_DIR)
    
    if not base_file:
        print(f"❌ Base 모델 폴더({BASE_MODEL_DIR})에서 가중치 파일을 찾을 수 없습니다.")
        return
    if not target_file:
        print(f"❌ Target 모델 폴더({TARGET_MODEL_DIR})에서 가중치 파일을 찾을 수 없습니다.")
        return

    # 로드
    base_keys, base_dict = load_keys(base_file)
    target_keys, target_dict = load_keys(target_file)

    if base_keys is None or target_keys is None:
        print("❌ 가중치 로드에 실패하여 분석을 중단합니다.")
        return

    # [비교 로직은 동일]
    missing_in_target = base_keys - target_keys
    extra_in_target = target_keys - base_keys

    print(f"\n📊 요약:")
    print(f"   - Base 파일: {os.path.basename(base_file)} ({len(base_keys)} keys)")
    print(f"   - Target 파일: {os.path.basename(target_file)} ({len(target_keys)} keys)")
    
    # 불필요한 키 (Extra Keys) - 에러의 주범
    if extra_in_target:
        print(f"\n🚨 [Extra Keys] Target에만 존재하는 불필요한 키 ({len(extra_in_target)}개):")
        sorted_extras = sorted(list(extra_in_target))
        for i, k in enumerate(sorted_extras):
            if i < 20: 
                # shape 정보 출력 시도
                shape = "Unknown"
                if hasattr(target_dict[k], 'shape'): shape = target_dict[k].shape
                elif isinstance(target_dict[k], dict): shape = "Dict?" # safetensors numpy 등
                else: shape = getattr(target_dict[k], 'shape', 'Unknown')
                
                print(f"   ➕ {k} | Shape: {shape}")
            else:
                print(f"   ... (총 {len(extra_in_target)}개)")
                break
        
        # 분석 메시지
        if any("bias" in k for k in extra_in_target):
            print("\n   👉 [진단] 'bias'가 포함된 키가 다수 발견되었습니다.")
            print("       -> LoRA 병합 시 원래 bias=False인 레이어에 bias가 생성된 것으로 보입니다.")
            print("       -> 'fix_checkpoint.py'를 사용하여 이 키들을 삭제해야 합니다.")
            
    elif not missing_in_target:
        print("\n✅ 키 목록이 완벽하게 일치합니다!")
    
    # 누락된 키
    if missing_in_target:
        print(f"\n⚠️ [Missing Keys] Target에서 사라진 키 ({len(missing_in_target)}개):")
        for k in list(missing_in_target)[:10]:
            print(f"   ➖ {k}")

if __name__ == "__main__":
    analyze_diff()

🔍 분석 시작
📂 파일 로딩 시도: PaddlePaddle/PaddleOCR-VL/model.safetensors
📂 파일 로딩 시도: ./PaddleOCR-VL-SFT-test06/model_state.pdparams

📊 요약:
   - Base 파일: model.safetensors (620 keys)
   - Target 파일: model_state.pdparams (620 keys)

✅ 키 목록이 완벽하게 일치합니다!


In [1]:
import paddle
from ernie.modeling_paddleocr_vl import PaddleOCRVLForConditionalGeneration
from ernie.tokenizer import Ernie4_5_Tokenizer
from data_processor.image_preprocessor.image_preprocessor_siglip import SiglipImageProcessor
from paddleformers.peft import LoRAModel

#BASE = "PaddlePaddle/PaddleOCR-VL"                     # 학습 베이스와 동일
BASE = "/home/mango/ERNIE/PaddleOCR-VL-SFT-paddle-merged"     # LoRA 출력 디렉터리(ERNIE 포맷)

# 1) 베이스 로드(HF->Paddle 변환 필요 시)
model = PaddleOCRVLForConditionalGeneration.from_pretrained(BASE)
tokenizer = Ernie4_5_Tokenizer.from_pretrained(BASE, padding_side="right", model_max_length=4096)
image_preprocess = SiglipImageProcessor.from_pretrained(BASE)

/root/anaconda3/envs/deepseek-ocr/lib/python3.12/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
W1125 10:11:55.174090 11679 gpu_resources.cc:114] Please NOTE: device: 0, GPU Compute Capability: 8.6, Driver API Version: 12.6, Runtime API Version: 11.8
[2025-11-25 10:11:56,726] [    INFO] - Using download source: huggingface
[2025-11-25 10:11:56,727] [    INFO] - Loading configuration file /home/mango/ERNIE/PaddleOCR-VL-SFT-paddle-merged/config.json
[2025-11-25 10:11:56,728] [    INFO] - Loading weights file /home/mango/ERNIE/PaddleOCR-VL-SFT-paddle-merged/model_state.pdparams
[2025-11-25 10:11:59,269] [    INFO] - Loaded weights file from disk, setting weights to model.
[2025-11-25 10:11:59,438] [    INFO] - use GQA - num_heads: 16- num_key_

In [3]:
model

PaddleOCRVLForConditionalGeneration(
  (mlp_AR): Projector(
    (pre_norm): LayerNorm(normalized_shape=[1152], epsilon=1e-05)
    (linear_1): Linear(in_features=4608, out_features=4608, dtype=bfloat16)
    (act): GELUActivation()
    (linear_2): Linear(in_features=4608, out_features=1024, dtype=bfloat16)
  )
  (visual): SiglipVisionModel(
    (vision_model): SiglipVisionTransformer(
      (embeddings): SiglipVisionEmbeddings(
        (patch_embedding): Conv2D(3, 1152, kernel_size=[14, 14], stride=[14, 14], padding=VALID, data_format=NCHW)
        (position_embedding): Embedding(729, 1152, sparse=False, scale_grad_by_freq=False, dtype=paddle.bfloat16)
        (packing_position_embedding): Embedding(32768, 1152, sparse=False, scale_grad_by_freq=False, dtype=paddle.bfloat16)
      )
      (encoder): SiglipEncoder(
        (layers): LayerList(
          (0): SiglipEncoderLayer(
            (layer_norm1): LayerNorm(normalized_shape=[1152], epsilon=1e-06)
            (self_attn): SiglipAtten

In [2]:
model

PaddleOCRVLForConditionalGeneration(
  (mlp_AR): Projector(
    (pre_norm): LayerNorm(normalized_shape=[1152], epsilon=1e-05)
    (linear_1): Linear(in_features=4608, out_features=4608, dtype=bfloat16)
    (act): GELUActivation()
    (linear_2): Linear(in_features=4608, out_features=1024, dtype=bfloat16)
  )
  (visual): SiglipVisionModel(
    (vision_model): SiglipVisionTransformer(
      (embeddings): SiglipVisionEmbeddings(
        (patch_embedding): Conv2D(3, 1152, kernel_size=[14, 14], stride=[14, 14], padding=VALID, data_format=NCHW)
        (position_embedding): Embedding(729, 1152, sparse=False, scale_grad_by_freq=False, dtype=paddle.bfloat16)
        (packing_position_embedding): Embedding(32768, 1152, sparse=False, scale_grad_by_freq=False, dtype=paddle.bfloat16)
      )
      (encoder): SiglipEncoder(
        (layers): LayerList(
          (0): SiglipEncoderLayer(
            (layer_norm1): LayerNorm(normalized_shape=[1152], epsilon=1e-06)
            (self_attn): SiglipAtten

In [2]:
from vllm import LLM
llm = LLM(model="home/mango/ERNIE/PaddleOCR-VL-SFT-paddle-merged")


INFO 11-24 18:20:59 [__init__.py:239] Automatically detected platform cuda.
INFO 11-24 18:21:01 [__init__.py:30] Available plugins for group vllm.general_plugins:
INFO 11-24 18:21:01 [__init__.py:32] name=register_paddlex_genai_models, value=paddlex.inference.genai.backends.vllm:register_models
INFO 11-24 18:21:01 [__init__.py:34] all available plugins for group vllm.general_plugins will be loaded.
INFO 11-24 18:21:01 [__init__.py:36] set environment variable VLLM_PLUGINS to control which plugins to load.
INFO 11-24 18:21:05 [__init__.py:44] plugin register_paddlex_genai_models loaded.


ImportError: cannot import name 'AttrsDescriptor' from 'triton.compiler.compiler' (/root/anaconda3/envs/deepseek-ocr/lib/python3.12/site-packages/triton/compiler/compiler.py)

In [1]:
import os
import paddle
from PIL import Image

from ernie.modeling_paddleocr_vl import PaddleOCRVLForConditionalGeneration
from ernie.tokenizer import Ernie4_5_Tokenizer
from data_processor.image_preprocessor.image_preprocessor_siglip import SiglipImageProcessor
from paddleformers.peft import LoRAModel

# 1) 경로
BASE = "PaddlePaddle/PaddleOCR-VL"  # 학습 시 사용한 베이스와 동일
LORA = "/home/mango/ERNIE/PaddleOCR-VL-SFT-test05"  # peft_model-*.safetensors + lora_config.json

# 2) 베이스 모델/토크나이저/이미지 프로세서 로드
#    HF 허브 베이스에서 Paddle로 변환 로드가 필요하면 convert_from_hf=True
model = PaddleOCRVLForConditionalGeneration.from_pretrained(BASE, convert_from_hf=True)
tokenizer = Ernie4_5_Tokenizer.from_pretrained(BASE, padding_side="right", model_max_length=8192)
image_processor = SiglipImageProcessor.from_pretrained(BASE)

# 3) LoRA 어댑터 로드(ERNIE/Paddle 포맷 지원)
model = LoRAModel.from_pretrained(model=model, lora_path=LORA)
model.eval()

# 4) 입력 준비 (예: OCR 프롬프트)
img_path = "img-000000012.jpg"
image = Image.open(img_path).convert("RGB")

messages = [
    {"role": "user",
     "content": [
        {"type": "image", "image": image},
        {"type": "text",  "text": "OCR:"}
     ]}
]

# 텍스트 토크나이즈
txt_inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_dict=True,
    return_tensors="pd"
)

# 비전 전처리: pixel_values + image_grid_thw 필수
vis_inputs = image_processor.preprocess(images=[image], return_tensors="pd")

# 모델 입력 합치기
inputs = {**txt_inputs, **vis_inputs}

# 5) 생성
# 중요: transformers처럼 **inputs 로 호출하면 TypeError가 납니다.
#       PaddleOCR-VL은 inputs=dict 를 첫 인자로 넘겨야 합니다.
gen_ids = model.generate(inputs, max_new_tokens=1024, use_cache=True)[0]

# 6) 디코딩
text = tokenizer.decode(gen_ids.numpy().tolist(), skip_special_tokens=True)
print(text)

/root/anaconda3/envs/deepseek-ocr/lib/python3.12/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
W1124 17:42:38.978982 254112 gpu_resources.cc:114] Please NOTE: device: 0, GPU Compute Capability: 8.6, Driver API Version: 12.6, Runtime API Version: 11.8
[2025-11-24 17:42:41,147] [    INFO] - Using download source: huggingface
[2025-11-24 17:42:41,148] [    INFO] - Loading configuration file PaddlePaddle/PaddleOCR-VL/config.json
[2025-11-24 17:42:41,151] [    INFO] - Loading weights file PaddlePaddle/PaddleOCR-VL/model.safetensors
[2025-11-24 17:42:45,577] [    INFO] - Loaded weights file from disk, setting weights to model.
[2025-11-24 17:42:45,733] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-24 17:42:45,739] [    IN

ValueError: Image features and image tokens do not match: tokens: 0, features 390

In [ ]:
from PIL import Image
import torch
import paddle
from ernie.modeling_paddleocr_vl import PaddleOCRVLForConditionalGeneration
from ernie.tokenizer import Ernie4_5_Tokenizer
from data_processor.image_preprocessor.image_preprocessor_siglip import SiglipImageProcessor
from paddleformers.peft import LoRAModel
from paddlex.inference.models.doc_vlm.processors.paddleocr_vl._paddleocr_vl import PaddleOCRVLProcessor
from data_processor.image_preprocessor.image_preprocessor_siglip import SiglipImageProcessor
from ernie.tokenizer import Ernie4_5_Tokenizer  # paddlex 배포 토크나이저가 HF 포맷

CHOSEN_TASK = "ocr"  # Options: 'ocr' | 'table' | 'chart' | 'formula'

PROMPTS = {
    "ocr": "OCR:",
    "table": "Table Recognition:",
    "formula": "Formula Recognition:",
    "chart": "Chart Recognition:",
}

image_path = "000000014_경남.jpg"
image = Image.open(image_path).convert("RGB")


model_dir = "./PaddleOCR-VL-SFT-test02"  # --vl_rec_model_dir와 동일한 구조
image_processor = SiglipImageProcessor.from_pretrained(model_dir)
tokenizer = Ernie4_5_Tokenizer.from_pretrained(model_dir, trust_remote_code=True)

processor = PaddleOCRVLProcessor(image_processor=image_processor, tokenizer=tokenizer)

messages = [
    {"role": "user",         
     "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": PROMPTS[CHOSEN_TASK]},
        ]
    }
]

inputs = processor.tokenizer.apply_chat_template(
    messages, 
    tokenize=True, 
    add_generation_prompt=True, 	
    return_dict=True,
    return_tensors="pd"
)


outputs = model.generate(**inputs, max_new_tokens=1024)
outputs = processor.batch_decode(outputs, skip_special_tokens=True)[0]
print(outputs)


TypeError: PaddleOCRVLForConditionalGeneration.generate() missing 1 required positional argument: 'inputs'

{'input_ids': Tensor(shape=[1, 59], dtype=int64, place=Place(gpu:0), stop_gradient=True,
       [[93988, 23374, 727  , 1708 , 468  , 2580 , 574  , 468  , 2580 , 1708 ,
         518  , 1320 , 93966, 93937, 2848 , 93937, 2848 , 3003 , 2943 , 93950,
         13079, 1421 , 93950, 12   , 7    , 93960, 7    , 10   , 488  , 93919,
         3    , 93960, 10   , 9    , 17785, 6    , 11   , 12790, 11   , 8    ,
         93976, 3    , 93951, 745  , 11722, 727  , 1708 , 468  , 679  , 574  ,
         468  , 679  , 1708 , 468  , 93972, 2497 , 11051, 17547, 93919]]), 'position_ids': Tensor(shape=[1, 59], dtype=int64, place=Place(gpu:0), stop_gradient=True,
       [[0 , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
         36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
         54, 55, 56, 57, 58]]), 'attention_mask': Tensor(shape=[1, 1, 59, 59], dtype=int64, place=Place(gpu

In [1]:
import paddle
from types import SimpleNamespace
from PIL import Image

from ernie.modeling_paddleocr_vl import PaddleOCRVLForConditionalGeneration
from ernie.tokenizer import Ernie4_5_Tokenizer
from paddleformers.peft import LoRAModel

from data_processor.steps.end2end_processing.processor import End2EndProcessor
from data_processor.image_preprocessor.image_preprocessor_siglip import SiglipImageProcessor

BASE = "PaddlePaddle/PaddleOCR-VL"   # 학습에 사용한 베이스
LORA = "/home/mango/ERNIE/PaddleOCR-VL-SFT-test02"  # LoRA 디렉토리
IMG = "/000000014_경남.jpg"

# 1) 모델 로드 (Paddle/ERNIE)
model = PaddleOCRVLForConditionalGeneration.from_pretrained(BASE, convert_from_hf=True)
model = LoRAModel.from_pretrained(model=model, lora_path=LORA)
model.eval()

# 2) 프로세서(ERNIE 네이티브)
tokenizer = Ernie4_5_Tokenizer.from_pretrained(LORA, padding_side="right", model_max_length=4096)
image_preprocess = SiglipImageProcessor.from_pretrained(LORA)

args = SimpleNamespace(
    # End2EndProcessor/하위 단계들이 요구하는 최소 필드
    batch_size=1,
    load_args_from_api=False,
    variable_resolution=1,
    min_pixels=28*28*130,
    max_pixels=28*28*1280,
    spatial_conv_size=1,
    im_prefix_length=64,
    max_seq_length=4096,
    sft_shift_by_one=False,
    sft_replace_ids=True,
    sft_image_rescale=True,
    sft_image_normalize=True,
    image_token_len=64,
    image_dtype="float32",
    render_timestamp=False,
    chat_template="ernie_vl",
    use_pic_id=False,
    drop_untrainble_sample=True,
    video_min_pixels=28*28*130,
    video_max_pixels=28*28*1280,
    corpus_name="inference",
    one_sample_in_one_seq=True,
    adaptive_max_imgtoken_option="64,96,128",
    adaptive_max_imgtoken_rate="0.6,0.3,0.1",
)

processor = End2EndProcessor(args, tokenizer, image_preprocess)

# 3) 입력 구성 (채팅 템플릿은 내부에서 적용됨)
img = Image.open(IMG).convert("RGB")
user_input = {
    "context": [
        {
            "role": "user",
            "utterance": [
                {"type": "text", "text": "OCR:"},
                {"type": "image_url", "image_url": {"url": img}},
            ],
        }
    ]
}

# 4) 텐서 생성
rets = processor.process(user_input)
ret = rets[0]

# 학습 루프와 동일한 키로 변환
inputs = {
    "input_ids": paddle.to_tensor(ret["input_ids"], dtype="int64").unsqueeze(0),
    "token_type_ids": paddle.to_tensor(ret["token_type_ids"], dtype="int64").unsqueeze(0),
    "image_type_ids": paddle.to_tensor(ret["image_type_ids"], dtype="int64").unsqueeze(0),
    "pixel_values": paddle.to_tensor(ret["images"], dtype="float32"),
    "image_grid_thw": paddle.to_tensor(ret["grid_thw"], dtype="int64"),
    "position_ids": paddle.to_tensor(ret["position_ids"], dtype="int64"),
}

# 학습 루프의 position_ids 변환 적용
position_ids = inputs["position_ids"].squeeze(0).transpose([1, 0]).unsqueeze(1)
inputs["position_ids"] = position_ids

# 5) 생성 및 디코딩
gen_ids = model.generate(**inputs, max_new_tokens=512, use_cache=True)[0]
text = tokenizer.decode(gen_ids.numpy().tolist(), skip_special_tokens=True)
print(text)

/root/anaconda3/envs/deepseek-ocr/lib/python3.12/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
W1111 13:59:25.386947  2271 gpu_resources.cc:114] Please NOTE: device: 0, GPU Compute Capability: 8.6, Driver API Version: 12.6, Runtime API Version: 11.8
[2025-11-11 13:59:28,203] [    INFO] - Using download source: huggingface
[2025-11-11 13:59:28,203] [    INFO] - Loading configuration file PaddlePaddle/PaddleOCR-VL/config.json
[2025-11-11 13:59:28,205] [    INFO] - Loading weights file PaddlePaddle/PaddleOCR-VL/model.safetensors
[2025-11-11 13:59:32,475] [    INFO] - Loaded weights file from disk, setting weights to model.
[2025-11-11 13:59:32,679] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-11 13:59:32,684] [    INF

AttributeError: 'types.SimpleNamespace' object has no attribute 'video_fps'

In [1]:
from transformers import AutoProcessor

model_path = "./PaddleOCR-VL-SFT-test02"
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)

OSError: ./PaddleOCR-VL-SFT-test02 does not appear to have a file named processing_ppocrvl.py. Checkout 'https://huggingface.co/./PaddleOCR-VL-SFT-test02/tree/main' for available files.

In [ ]:
from paddleocr import LayoutDetection

#이미지 리스트로
image_list = ["img-000000012.jpg", "img-000000000.jpg", "img-000000022.jpg"]
model = LayoutDetection(model_name="PP-DocLayoutV2", device="gpu")
output = model.predict(image_list, batch_size=2, layout_nms=True, threshold=0.5)
for res in output:
    res.print()
    res.save_to_img(save_path="./output/")
    res.save_to_json(save_path="./output/res.json")


Model files already exist. Using cached files. To redownload, please delete the directory manually: `/root/.paddlex/official_models/PP-DocLayoutV2`.
{'res': {'input_path': 'img-000000012.jpg', 'page_index': None, 'boxes': [{'cls_id': 22, 'label': 'text', 'score': 0.5718106031417847, 'coordinate': [0.15052554, 0.18856022, 1078.6244, 282.74127]}]}}
{'res': {'input_path': 'img-000000000.jpg', 'page_index': None, 'boxes': [{'cls_id': 7, 'label': 'figure_title', 'score': 0.5411680936813354, 'coordinate': [629.2026, 127.50459, 1020.1976, 179.55023]}, {'cls_id': 21, 'label': 'table', 'score': 0.9869638085365295, 'coordinate': [69.662895, 110.67651, 1577.9956, 2223.0688]}]}}
{'res': {'input_path': 'img-000000022.jpg', 'page_index': None, 'boxes': [{'cls_id': 17, 'label': 'paragraph_title', 'score': 0.8050800561904907, 'coordinate': [14.97492, 7.118814, 545.42346, 384.38373]}]}}


: 

In [ ]:
from paddleocr import TableCellsDetection
model = TableCellsDetection(model_name="RT-DETR-L_wired_table_cell_det")
output = model.predict("img-000000000.jpg", threshold=0.3, batch_size=1)
for res in output:
    res.print(json_format=False)
    res.save_to_img("./output/")
    res.save_to_json("./output/res.json")

/root/anaconda3/envs/myenv/lib/python3.9/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/root/.paddlex/official_models/RT-DETR-L_wired_table_cell_det`.
{'res': {'input_path': 'img-000000012.jpg', 'page_index': None, 'boxes': [{'cls_id': 0, 'label': 'cell', 'score': 0.4862610399723053, 'coordinate': [1.878852, 90.55357, 1079, 190.20425]}, {'cls_id': 0, 'label': 'cell', 'score': 0.46319085359573364, 'coordinate': [3.82787, 3.0447578, 1076.659, 90.90404]}, {'cls_id': 0, 'label': 'cell', 'score': 0.44285157322883606, 'coordinate': [187.59222, 189.90685, 1076.9583, 281.95325]}, {'cls_id': 0, 'label': 'cell', 'score': 0.41178473830223083, 'coordin

In [ ]:
from paddleocr import PaddleOCR
image_list = ["img-000000012.jpg", "img-000000000.jpg", "img-000000022.jpg"]
ocr = PaddleOCR(use_angle_cls=False, lang='korean')
output = ocr.predict(image_list)
#이미지 저장
for res in output:
    res.print()
    res.save_to_img("output")
    res.save_to_json("output")

/tmp/ipykernel_50813/3833193561.py:3: DeprecationWarning: The parameter `use_angle_cls` has been deprecated and will be removed in the future. Please use `use_textline_orientation` instead.
  ocr = PaddleOCR(use_angle_cls=False, lang='korean')
Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/root/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/root/.paddlex/official_models/UVDoc`.
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/root/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('korean_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/root/.paddlex/official_mo

: 

In [5]:
output[0]['doc_preprocessor_res']['angle']

180

In [ ]:
output[0]['rec_scores']

[0.9997122287750244, 0.9724054336547852]

: 

In [2]:
from paddlex.inference import list_available_plugins
print(list_available_plugins())


ImportError: cannot import name 'list_available_plugins' from 'paddlex.inference' (/root/anaconda3/envs/myenv/lib/python3.9/site-packages/paddlex/inference/__init__.py)